In [1]:
import torch
import numpy as np
import pandas as pd

from botorch.utils.sampling import draw_sobol_samples
from bonp.core import optimize_acqf_and_get_recommendation

tkwargs = {
    "dtype": torch.float64,
    # "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "device": torch.device("cpu"),
}

torch.manual_seed(0)

# Set the aiming particle size (um)
TARGET_SIZE = torch.tensor(3.0, **tkwargs)

# Set the solvent dictionary: 0 for DMAc, 1 for CHCl3
SOLVENT_DICT = {1: "CHCl3", 0: "DMAc"}

# Set the bounds for the processing parameters
bounds = torch.tensor([[0.05, 0.01, 10, 0], [5.0, 60.0, 18, 1]], **tkwargs)

# Set params for guided experimental design
BATCH_SIZE = 2
strategy = "qEICF_vi_mixed_con"

print(f"Performing Bayesian optimization using {strategy}...")

Performing Bayesian optimization using qEICF_vi_mixed_con...


# Round 0: Sobel Init

In [2]:
# log transform the flow rate
bounds[0, 1] = torch.log(bounds[0, 1])
bounds[1, 1] = torch.log(bounds[1, 1])

# Generate sobel sequence for initial data
X_init = draw_sobol_samples(bounds=bounds, n=8, q=1).squeeze(1)

# If the flow rate is log-transformed, transform the flow rate back to the original scale
X_init[:, 1] = torch.exp(X_init[:, 1])
bounds[0, 1] = torch.exp(bounds[0, 1])
bounds[1, 1] = torch.exp(bounds[1, 1])

# round up the solvent dimension to get the integer
X_init[:, 3] = torch.round(X_init[:, 3])

# Initialize the experiment record
experiment_df = pd.DataFrame(
    X_init.numpy(),
    columns=["Concentration (%)", "Flow Rate (uL/min)", "Voltage (kV)", "Solvent"],
)
# Convert the solvent column to string
experiment_df["Solvent"] = experiment_df["Solvent"].map({1: "CHCl3", 0: "DMAc"})
experiment_df

,Concentration (%),Flow Rate (uL/min),Voltage (kV),Solvent
0,2.401781,1.732388,13.955834,DMAc
1,4.062639,0.435760,15.700379,CHCl3
2,2.880187,49.111354,11.789306,DMAc
3,0.755436,0.010943,17.554481,CHCl3
4,0.112274,10.427114,14.522246,CHCl3
5,3.552020,0.059013,12.758979,DMAc
6,4.550922,2.394492,16.730707,CHCl3
7,1.884746,0.209578,10.988069,DMAc


In [3]:
# Add sobel init results from experiment
Y_init = torch.tensor(
    [
        [0.56, 1.00],
        [1.0, 0.00],
        [15.0, 0.00],
        [1.2, 0.00],
        [6.26, 1.00],
        [0.15, 1.00],
        [5.24, 1.00],
        [1.12, 1.00],
    ],
    **tkwargs
)

## Round 1:

In [4]:
Y_distance = -np.abs(Y_init[:, 0] - TARGET_SIZE)
best_Y_ind = np.argmax(np.ma.masked_array(Y_distance, mask=~Y_init[:, 1].bool()))
raw_Y = Y_init[best_Y_ind].numpy()
print(f"Current best result:{raw_Y[0]} um")

# perform the optimization and recommend next experiments
new_x = optimize_acqf_and_get_recommendation(
    X_init,
    Y_init,
    bounds,
    y_target=TARGET_SIZE,
    strategy=strategy,
    batch_size=BATCH_SIZE,
)
# Initialize the experiment record
new_experiment_df = pd.DataFrame(
    new_x.numpy(),
    columns=["Concentration (%)", "Flow Rate (uL/min)", "Voltage (kV)", "Solvent"],
)
# Convert the solvent column to string
new_experiment_df["Solvent"] = new_experiment_df["Solvent"].map({1: "CHCl3", 0: "DMAc"})
new_experiment_df

Current best result:1.12 um


,Concentration (%),Flow Rate (uL/min),Voltage (kV),Solvent
0,1.660193,0.800621,10.740816,DMAc
1,0.364120,3.647617,14.629043,CHCl3


In [5]:
# Perform the experiment and obtain the particle size for new_x
new_y = torch.tensor([[0.30, 1], [2.69, 1]], **tkwargs)

# Round 2:

In [6]:
# combine X_init and new_x
X_init = torch.cat((X_init, new_x))
Y_init = torch.cat((Y_init, new_y))

# print the current best result
Y_distance = -np.abs(Y_init[:, 0] - TARGET_SIZE)
best_Y_ind = np.argmax(np.ma.masked_array(Y_distance, mask=~Y_init[:, 1].bool()))
raw_Y = Y_init[best_Y_ind].numpy()
print(f"Current best result:{raw_Y[0]} um")

# perform the optimization and recommend next experiments
new_x = optimize_acqf_and_get_recommendation(
    X_init,
    Y_init,
    bounds,
    y_target=TARGET_SIZE,
    strategy=strategy,
    batch_size=BATCH_SIZE,
)
# Initialize the experiment record
new_experiment_df = pd.DataFrame(
    new_x.numpy(),
    columns=["Concentration (%)", "Flow Rate (uL/min)", "Voltage (kV)", "Solvent"],
)
# Convert the solvent column to string
new_experiment_df["Solvent"] = new_experiment_df["Solvent"].map({1: "CHCl3", 0: "DMAc"})
new_experiment_df

Current best result:2.69 um


,Concentration (%),Flow Rate (uL/min),Voltage (kV),Solvent
0,0.57426,3.740271,16.473412,CHCl3
1,0.05000,3.554317,14.514592,CHCl3


In [7]:
# Perform the experiment and obtain the particle size for new_x
new_y = torch.tensor([[4.69, 1], [10.64, 0]], **tkwargs)

# Round 3:

In [8]:
# combine X_init and new_x
X_init = torch.cat((X_init, new_x))
Y_init = torch.cat((Y_init, new_y))

# print the current best result
Y_distance = -np.abs(Y_init[:, 0] - TARGET_SIZE)
best_Y_ind = np.argmax(np.ma.masked_array(Y_distance, mask=~Y_init[:, 1].bool()))
raw_Y = Y_init[best_Y_ind].numpy()
print(f"Current best result:{raw_Y[0]} um")

# perform the optimization and recommend next experiments
new_x = optimize_acqf_and_get_recommendation(
    X_init,
    Y_init,
    bounds,
    y_target=TARGET_SIZE,
    strategy=strategy,
    batch_size=BATCH_SIZE,
)
# Initialize the experiment record
new_experiment_df = pd.DataFrame(
    new_x.numpy(),
    columns=["Concentration (%)", "Flow Rate (uL/min)", "Voltage (kV)", "Solvent"],
)
# Convert the solvent column to string
new_experiment_df["Solvent"] = new_experiment_df["Solvent"].map({1: "CHCl3", 0: "DMAc"})
new_experiment_df

Current best result:2.69 um


,Concentration (%),Flow Rate (uL/min),Voltage (kV),Solvent
0,1.628817,1.917756,16.197772,CHCl3
1,4.510426,1.384216,14.890315,CHCl3


In [9]:
# Perform the experiment and obtain the particle size for new_x
new_y = torch.tensor([[4.14, 1], [4.05, 1]], **tkwargs)

# Round 4

In [10]:
# combine X_init and new_x
X_init = torch.cat((X_init, new_x))
Y_init = torch.cat((Y_init, new_y))

# print the current best result
Y_distance = -np.abs(Y_init[:, 0] - TARGET_SIZE)
best_Y_ind = np.argmax(np.ma.masked_array(Y_distance, mask=~Y_init[:, 1].bool()))
raw_Y = Y_init[best_Y_ind].numpy()
print(f"Current best result:{raw_Y[0]} um")

# perform the optimization and recommend next experiments
new_x = optimize_acqf_and_get_recommendation(
    X_init,
    Y_init,
    bounds,
    y_target=TARGET_SIZE,
    strategy=strategy,
    batch_size=BATCH_SIZE,
)
# Initialize the experiment record
new_experiment_df = pd.DataFrame(
    new_x.numpy(),
    columns=["Concentration (%)", "Flow Rate (uL/min)", "Voltage (kV)", "Solvent"],
)
# Convert the solvent column to string
new_experiment_df["Solvent"] = new_experiment_df["Solvent"].map({1: "CHCl3", 0: "DMAc"})
new_experiment_df

Current best result:2.69 um


,Concentration (%),Flow Rate (uL/min),Voltage (kV),Solvent
0,4.445068,1.296518,17.386783,CHCl3
1,4.020724,1.078188,16.275159,CHCl3


In [11]:
# Perform the experiment and obtain the particle size for new_x
new_y = torch.tensor([[3.58, 1], [3.29, 1]], **tkwargs)